In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '4'

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
from os import path

from torch import nn
from torch.nn import functional as F
from torch.nn.functional import scaled_dot_product_attention
from torch.nn.attention import SDPBackend, sdpa_kernel

from tqdm import tqdm
from einops import rearrange
from torch.optim import AdamW

import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from omegaconf import OmegaConf, open_dict
from experanto.datasets import ChunkDataset, SimpleChunkedDataset
from experanto.utils import LongCycler, MultiEpochsDataLoader
from experanto.dataloaders import get_multisession_dataloader
from torch.nn.attention import SDPBackend, sdpa_kernel

In [3]:
paths = ['dynamic29513-3-5-Video-full',
         'dynamic29514-2-9-Video-full',
         'dynamic29755-2-8-Video-full',
         'dynamic29647-19-8-Video-full',
         'dynamic29156-11-10-Video-full',
         'dynamic29623-4-9-Video-full',
         'dynamic29515-10-12-Video-full',
         'dynamic29234-6-9-Video-full',
         'dynamic29712-5-9-Video-full',
         'dynamic29228-2-10-Video-full'
        ]
full_paths = [path.join("/data/mouse_polly/", f) for f in paths]

In [4]:
from experanto.configs import DEFAULT_CONFIG as cfg
cfg.dataset.global_chunk_size = 16
cfg.dataset.global_sampling_rate = 8
cfg.dataset.modality_config.screen.include_blanks=True
cfg.dataset.modality_config.screen.valid_condition = {"tier": "train"}

cfg.dataloader.num_workers=4
cfg.dataloader.prefetch_factor=1
cfg.dataloader.batch_size=16
cfg.dataloader.pin_memory=True

In [5]:
# the multiple dataloader is an iterator that returns a tuple of (key, batch)
train_dl = get_multisession_dataloader(full_paths, cfg)

In [7]:
from hiera import Hiera


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
!wget https://dl.fbaipublicfiles.com/hiera/mae_hiera_huge_16x224.pth

--2024-10-15 23:21:37--  https://dl.fbaipublicfiles.com/hiera/mae_hiera_huge_16x224.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 99.84.238.206, 99.84.238.162, 99.84.238.154, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|99.84.238.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8531176921 (7.9G) [binary/octet-stream]
Saving to: ‘mae_hiera_huge_16x224.pth’

mae_hiera_huge_16x2 100%[===================>]   7.94G  25.6MB/s    in 5m 40s  

2024-10-15 23:27:18 (23.9 MB/s) - ‘mae_hiera_huge_16x224.pth’ saved [8531176921/8531176921]



In [8]:
model = Hiera.from_pretrained("facebook/hiera_huge_16x224.mae_in1k") # just mae pt, no ft


RepositoryNotFoundError: 401 Client Error. (Request ID: Root=1-670ef8de-758b1d334eb83ca45243e1af;9df36bf7-9dac-42e6-82c2-6ef8a7af6200)

Repository Not Found for url: https://huggingface.co/facebook/hiera_huge_16x224.mae_in1k/resolve/main/model.safetensors.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.
Invalid username or password.

In [ ]:
@pretrained_model({
    "mae_k400_ft_k400": "https://dl.fbaipublicfiles.com/hiera/hiera_huge_16x224.pth",
    "mae_k400": "https://dl.fbaipublicfiles.com/hiera/mae_hiera_huge_16x224.pth",
}, default="mae_k400_ft_k400")
def hiera_huge_16x224(**kwdargs):
    return hiera_base_16x224(
        embed_dim=256, num_heads=4, stages=(2, 6, 36, 4), **kwdargs
    )

In [ ]:
import torch
from hiera import Hiera
tiny_hiera = Hiera(input_size=(16,144, 256),
                     num_heads=4, 
                    embed_dim=256
                     stages=(2, 6,), 
                     q_pool=1, 
                     in_chans=1,
                     q_stride=(1, 2, 2),
                     mask_unit_size=(1, 8, 8),
                     patch_kernel=(3, 8, 8),
                     patch_stride=(2, 4, 4),
                     patch_padding=(1, 3, 3),
                     sep_pos_embed=True,)
#tiny_hiera = torch.compile(tiny_hiera).cuda().to(torch.bfloat16);